In [9]:
!pip install mysql-connector-python

In [9]:
import sys
import mysql.connector
from mysql.connector import Error


class class_sql_procedure:

    def sql_cursor_start(self):
        try:
            self.connection = mysql.connector.connect(host='localhost',
                                                 database='BankAccount',
                                                 user='root',
                                                 password='mysql')
            self.mycursor = self.connection.cursor()
            print('Mysql connection stablished')
        except :
            print(sys.exc_info()[0],"occured.")

    def sql_connection_close(self):
        try:
            if self.connection.is_connected():
                    self.mycursor.close()
                    self.connection.close()
                    print("MySQL connection is closed")
        except :
            print(sys.exc_info()[0],"occured.")

In [10]:
sql_obj = class_sql_procedure()

In [11]:
sql_obj.sql_cursor_start()

Mysql connection stablished


In [12]:
import mysql.connector
#create connection to mysql database
conn = mysql.connector.connect(user='root', password='mysql', host='localhost', database='BankAccount')

In [17]:
# create cursor object
cursor = conn.cursor()
# create table as User Table(User Id, User name, User dob,User email, User created date )


In [18]:
# create table as User Table(User Id, User name, User dob,User email, User created date )
cursor.execute("""CREATE TABLE IF NOT EXISTS User(
                User_Id INT AUTO_INCREMENT PRIMARY KEY, 
                User_Name VARCHAR(255) NOT NULL,
                User_DOB DATE NOT NULL,
                User_Email VARCHAR(255) NOT NULL,
                User_Created_Date DATE NOT NULL
                )""")

In [20]:
# insert data into table
cursor.execute('insert into User values(1, "John", "1990-01-01", "jhon@gamil.com", "2020-01-01")')
cursor.execute('insert into User values(2, "Smith", "1992-02-12", "smith@gmail.com", "2020-01-02")')
cursor.execute('insert into User values(3, "Peter", "1993-02-10", "Peter@gmail.com", "2020-01-04")')
cursor.execute('insert into User values(4, "James", "1996-02-11", "James@gmail.com", "2020-01-05")')

In [21]:
#create table as Bank_Account Table  ( User Id, bank account id, bank account number, is user active, amount )
cursor.execute("""CREATE TABLE IF NOT EXISTS Bank_Account(
                User_Id INT NOT NULL,
                Bank_Account_Id INT AUTO_INCREMENT PRIMARY KEY,
                Bank_Account_Number VARCHAR(255) NOT NULL,
                Is_User_Active BOOLEAN NOT NULL,
                Amount DECIMAL(10,2) NOT NULL,
                FOREIGN KEY (User_Id) REFERENCES User(User_Id)
                )""")
# insert data into Bank_Account table
cursor.execute('insert into Bank_Account values(1, 101, "123456781", 1, 10000)')
cursor.execute('insert into Bank_Account values(2, 102, "123456782", 1, 20000)')
cursor.execute('insert into Bank_Account values(3, 103, "123456783", 1, 30000)')
cursor.execute('insert into Bank_Account values(4, 104, "123456784", 1, 40000)')

In [22]:
#create table as Transaction Table ( Transaction date, User Id, Bank A/C id, withdrawn amount )
cursor.execute("""CREATE TABLE IF NOT EXISTS Transaction(
                Transaction_Date DATE NOT NULL,
                User_Id INT NOT NULL,
                Bank_Account_Id INT NOT NULL,
                Withdrawn_Amount DECIMAL(10,2) NOT NULL,
                FOREIGN KEY (User_Id) REFERENCES User(User_Id),
                FOREIGN KEY (Bank_Account_Id) REFERENCES Bank_Account(Bank_Account_Id)
                )""")
# insert data into Transaction table and date format is yyyy-mm-dd and same person having multiple random transactions
cursor.execute('insert into Transaction values("2020-01-01", 1, 101, 1000)')
cursor.execute('insert into Transaction values("2020-01-02", 1, 101, 2000)')
cursor.execute('insert into Transaction values("2020-01-03", 1, 101, 3000)')
cursor.execute('insert into Transaction values("2020-01-04", 2, 102, 4000)')
cursor.execute('insert into Transaction values("2020-01-05", 2, 102, 5000)')
cursor.execute('insert into Transaction values("2020-01-06", 3, 103, 6000)')
cursor.execute('insert into Transaction values("2020-01-07", 3, 103, 7000)')
cursor.execute('insert into Transaction values("2020-01-08", 4, 104, 8000)')
cursor.execute('insert into Transaction values("2020-01-09", 4, 104, 9000)')


In [84]:
# Create a new stored procedure in mysql to check the account balance of a particular user
cursor.execute("""
CREATE PROCEDURE Check_Account_Bal(IN UserId INT)
BEGIN
    SELECT SUM(Withdrawn_Amount) FROM Transaction WHERE User_Id = UserId;
END""")

In [89]:
cursor.callproc('Check_Account_Bal',[4,])
cursor.stored_results()

<generator object CMySQLCursor.stored_results at 0x00000231966FEEB0>

In [90]:
# print out the result
for result in cursor.stored_results():
            print(result.fetchone())

(Decimal('17000.00'),)


In [92]:
#create stored procedure to Check account statement: Users should be able to check all transactions for a given interval of time.(From date and To date)
cursor.execute("""CREATE PROCEDURE Acc_Statement(IN UserId INT, IN FromDate DATE, IN ToDate DATE)
BEGIN
SELECT * FROM Transaction WHERE User_Id = UserId AND Transaction_Date BETWEEN FromDate AND ToDate;
END""")

    

ProgrammingError: 1304 (42000): PROCEDURE Acc_Statement already exists

In [100]:
# call the stored procedure to check the account statement of a particular user
cursor.callproc('Acc_Statement', (1, '2020-01-01', '2020-01-10'))
# get the result
for result in cursor.stored_results():
    print(result.fetchall())

[(datetime.date(2020, 1, 1), 1, 101, Decimal('1000.00')), (datetime.date(2020, 1, 2), 1, 101, Decimal('2000.00')), (datetime.date(2020, 1, 3), 1, 101, Decimal('3000.00'))]


In [95]:
#Create a stored procedure to withdraw the amount where User must have a sufficient amount to withdraw money from his/her account.
# and Minimum balance 5000 rupees must be maintained in each user account.
cursor.execute("""CREATE PROCEDURE Withdraw_Amount(IN UserId INT, IN Amount DECIMAL(10,2))
BEGIN
SELECT * FROM Bank_Account WHERE User_Id = UserId AND Amount <= Amount AND Amount <= 5000;
END""")


ProgrammingError: 1304 (42000): PROCEDURE Withdraw_Amount already exists

In [98]:
# call the stored procedure to withdraw the amount where User must have a sufficient amount to withdraw money from his/her account.
# and Minimum balance 5000 rupees must be maintained in each user account.
cursor.callproc('Withdraw_Amount', (1, 10000))
# get the result
for result in cursor.stored_results():
    print(result.fetchall())

[]


In [102]:
cursor.execute("""CREATE PROCEDURE WithdrawAmount(IN UserId INT, IN Amount DECIMAL(10,2))
BEGIN
SELECT * FROM Bank_Account WHERE User_Id = UserId AND Amount >= 5000 AND Amount >= Amount - Withdrawn_Amount;
END""")
# call the stored `procedure to withdraw the amount when user has the amount greater than 5000 after reducing the amount by withdrawn amount
# and minimum balance 5000 rupees must be maintained in each user account.


In [111]:
cursor.callproc('Withdraw_Amount', (2, 4000))
# get the result
for result in cursor.stored_results():
    print(result.fetchall())

[(2, 102, '123456782', 1, Decimal('20000.00'))]


In [ ]:
cursor.execute("""CREATE PROCEDURE withdraw_amt01(IN User_Id1 INT)
Begin
SELECT distinct bank_account.User_Id,bank_account.Amount - sum(transaction.Withdrawn_Amount) as Current_Amt FROM Bank_Account  inner join Transaction on Bank_Account.User_Id = Transaction.User_Id                      
where bank_account.User_Id = User_Id1 group by bank_account.User_Id having Current_Amt > 5000;
End

OperationalError: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (PC)\nDB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (PC)\n')